In [ ]:
import sys
import os
import desc_bpz
cwd, _ = os.path.split("__file__")
parent_dir = os.path.abspath(os.path.join(cwd, ".."))
print(parent_dir)
os.environ["BPZDATAPATH"] = os.path.join(parent_dir, "DESC_BPZ", "src", "desc_bpz", "data_files")


In [ ]:
sys.float_info.min

## BPZ Tests

In [ ]:
cwd = os.path.abspath(os.path.split("__file__")[0])
parent_dir = os.path.join(cwd, "..")
# os.environ["LEPHAREDIR"] = os.path.join(parent_dir, "LEPHARE")
# os.environ["LEPHAREWORK"] = os.path.join(cwd)
os.environ["BPZDATAPATH"] = os.path.abspath(os.path.join(parent_dir, "DESC_BPZ", "src", "desc_bpz", "data_files"))
data_dir = os.environ.get("BPZDATAPATH", None)
fil_dir = None if data_dir is None else os.path.join(data_dir, "FILTER")
ab_dir = None if data_dir is None else os.path.join(data_dir, "AB")
sed_dir = None if data_dir is None else os.path.join(data_dir, "SED")
paths = [cwd, parent_dir, os.environ.get("BPZDATAPATH", None)]
for path in paths:
    print(path)

In [ ]:
import numpy as np
from desc_bpz.bpz_tools_py3 import prior_with_dict, prior

def test_old_vs_new_prior():
    zgrid = np.arange(0, 10.01, 0.1)
    mags = np.array([20.5, 21., 22.])
    # copy params needed for hdfn_gen prior
    paramdict = dict(fo_arr=[0.35, 0.5], kt_arr=[0.45, 0.147],
                     alpha_arr=[2.465, 1.806, 0.906],
                     zo_arr=[0.431, 0.390, 0.0626],
                     km_arr=[0.0913, 0.0636, 0.123],
                     m0=20.0,
                     numpertype=[1, 2, 3])

    for mag in mags:
        oldprior = prior(zgrid, mag, 'hdfn_gen')
        newprior = prior_with_dict(zgrid, mag, paramdict)
        assert (oldprior == newprior).all()
test_old_vs_new_prior()

In [ ]:
import os
import pytest
# dirname, _ = os.path.split(__file__)
# parent = os.path.join(dirname, '../src/desc_bpz/data_files/')


# This test has to go first 
def test_no_path():
    tmp = os.environ.get("BPZDATAPATH")
    if tmp is not None:
        del os.environ["BPZDATAPATH"]

    import desc_bpz.paths

    # Future proof in case a later test import the
    # module and are called first
    if desc_bpz.paths.data_dir is not None:
        # In this case first restore the env var
        if tmp is not None:
            os.environ["BPZDATAPATH"] = tmp
        # then mark the test as skipped
        pytest.skip("desc_bpz already imported by another test and path set")

    # should fail if BPZDATAPATH not set and we have not set manually
    with pytest.raises(RuntimeError):
        print(desc_bpz.paths.get_ab_file("El_B2004a.DC2LSST_g.AB"))

    # restore env var
    if tmp is not None:
        os.environ["BPZDATAPATH"] = tmp


def test_path():
    import desc_bpz.paths
    data_dir = os.environ.get("BPZDATAPATH", None)
    fil_dir = None if data_dir is None else data_dir + "/FILTER/"
    ab_dir = None if data_dir is None else data_dir + "/AB/"
    sed_dir = None if data_dir is None else data_dir + "/SED/"
    print(os.environ.get("BPZDATAPATH"))
    assert os.path.exists(desc_bpz.paths.get_ab_file("El_B2004a.DC2LSST_g.AB"))
    assert os.path.exists(desc_bpz.paths.get_fil_file("DC2LSST_g.res"))
    assert os.path.exists(desc_bpz.paths.get_sed_file("CWWSB4.list"))


test_no_path()
test_path()


In [ ]:
import os
import git

base_dir = os.path.abspath(git.Repo('.', search_parent_directories=True).working_tree_dir)
os.environ["BPZDATAPATH"] = os.path.abspath(os.path.join(base_dir, "DESC_BPZ", "src", "desc_bpz", "data_files"))
path = "$BPZDATAPATH"
print(os.path.expandvars(path))

## Roman Filter Transmission Data

In [ ]:
# import pandas as pd
# import numpy as np
# aperature = np.pi * 1.2**2
# df = pd.read_csv("Roman_effarea_20210614.csv").drop(columns=['SNPrism', 'Grism_1stOrder', 'Grism_0thOrder'])
# filters = ['F062', 'F087', 'F106', 'F129', 'F158', 'F184', 'F146', 'F213']
# df["Wave"] *= 10000 # convert from micron to angstrom units for BPZ
# df["Wave"] = df["Wave"].astype(int) # convert from micron to angstrom units
# print("Gains above one:")
# for filter in filters:
#     df[filter] = df[filter] / aperature # effective area to gain (1e-10 is needed to convert from angstrom to meters)
#     print(df[filter][df[filter]>1])

# for filter in filters:
#     df[["Wave", filter]].rename(columns={"Wave": "#Wavelength(AA)", filter: "Transmission"}).to_csv(f"data_files/RST_{filter}.res", sep=" ", index=False)

In [ ]:
# equivalencies = [ (u.Jy, u.erg/u.s/u.cm**2/u.AA, lambda x: 1e-33 * x / c.value , lambda x: c.value * x / 1e-33) ] 
# LSST_units = 1e-9*u.Jy
# flux_conversion = (LSST_units).to(u.erg/u.s/u.cm**2/u.AA, equivalencies=equivalencies)
# print(type(flux_conversion))

In [ ]:
import pandas as pd
# data sourced from https://roman.gsfc.nasa.gov/science/RRI/Roman_effarea_20210614.xlsx
df = pd.read_excel("Roman_effarea_20210614.xlsx", skiprows=1)
df.columns = [column.strip() for column in df.columns]
df = df.drop(columns=["SNPrism", "Grism_1stOrder", "Grism_0thOrder"])
print(df.columns)

In [ ]:
for column in df.columns:
    print(df[column].dtype)

In [11]:
import astropy.units as u
from astropy.constants import c, h
import pandas as pd
import numpy as np
from synphot import units, SourceSpectrum, SpectralElement, Observation
from synphot.models import Empirical1D
from tqdm import tqdm
from scipy.integrate import quad
import os
import desc_bpz

current_dir, _ = os.path.split("__file__")

bpz_dir, _ = os.path.split(desc_bpz.__file__)
sed_dir = os.path.join(bpz_dir, "data_files", "SED")

z_range = np.linspace(start=0.005, stop=3.505, num=int(3.5/0.01))
verbose = False


In [12]:
def load_filters() -> pd.DataFrame:
    """Loads filter transmission profiles

    Parameters
    ----------
    None

    Returns
    -------
    filters: pandas.DataFrame
        Filter transmission profiles as a function of wavelength
    """
    global current_dir
    filters = pd.read_excel(os.path.join(current_dir, "data_files", "FILTER", "Roman_effarea_20210614.xlsx"), skiprows=1)
    filters.columns = [column.strip() for column in filters.columns]
    if filters["Wave"][0] == 0:
        filters.drop(0, inplace=True)
    filters.drop(columns=["SNPrism", "Grism_1stOrder", "Grism_0thOrder"], inplace=True)
    filters["Wave"] *= 10000 # micron to angstrom conversion
    filters[filters.columns.drop("Wave")] /= np.pi * 1.2**2 # effective area to throughput conversion
    return filters

def spectral(wavelengths: np.ndarray) -> np.ndarray:
    """Converts wavelengths of arbitrary units into frequencies in units Hertz (Hz)

    Parameters
    ----------
    wavelenths: stropy.unit.quantity.Quantity
        Wavelengths of arbitrary units

    Returns
    -------
    frequencies: astropy.unit.quantity.Quantity
        Frequencies in units of Hertz (Hz)
    
    """
    frequencies = (wavelengths).to(u.Hz, equivalencies=u.spectral())
    return frequencies

filter_data = load_filters()
filter_ids = filter_data.columns.drop("Wave")
transmission_profiles, a_values, b_values = [], [], []
frequencies = spectral(filter_data["Wave"].to_numpy()*u.AA)
for filter_id in filter_ids:
    transmission_profiles.append(SpectralElement(Empirical1D, points=frequencies, lookup_table=filter_data[filter_id].to_numpy()))
    valid_indx = np.argwhere(filter_data[filter_id] > 0).flatten()
    a_values.append(np.min(filter_data["Wave"][valid_indx]))
    b_values.append(np.max(filter_data["Wave"][valid_indx]))
    assert(a_values[-1]<b_values[-1])

    # filter_data = []
    # filter_ids = []
    # for filter_file in tqdm(sorted([f for f in os.listdir(filter_dir) if "RST" in f]),
    #                         desc="Loading filter transmission profiles:",
    #                         disable=(not args.verbose), position=0):
    #     filter_ids.append(int(filter_file[filter_file.find("_")+2:filter_file.find(".")]))
    #     filter_data.append(SpectralElement.from_file(os.path.join(filter_dir, filter_file), data_start=1))
    


In [18]:
def magnitude(sed: SourceSpectrum, transmission_profile: SpectralElement, a: float, b: float) -> float:
    """Calculates AB calibrated magnitude 

    Parameters
    ----------
    sed: synphot.units.SourceSpectrum
        Spectral energy distribution

    transmission_profile: synphot.units.SpectralElement
        Transmission profile for a particular filter
    
    Returns
    -------
    m_AB: float
        AB calibrated magnitude
    """
    # observed_flux = quad(lambda f: transmission_profile(f).value * 3631 / (h.value  * f), a, b)[0]
    # reference_flux = quad(lambda f: transmission_profile(f).value * sed(f).value / (h.value * f), a, b)[0]
    # m_AB = 2.5*np.log10(observed_flux/reference_flux
    return 2.5*np.log10(quad(lambda f: transmission_profile(f).value * 3631 / (h.value  * f), a, b)[0]/quad(lambda f: transmission_profile(f).value * sed(f).value / (h.value * f), a, b)[0]) # m_AB

magnitudes = np.empty((8, len(z_range), len(transmission_profiles)))
for i, sed_file in enumerate([sed_file for sed_file in os.listdir(sed_dir) if "CWWSB4" not in sed_file]):
    galaxy_type = sed_file.rsplit('_', 1)[0]
    wavelength, flux = np.loadtxt(os.path.join(sed_dir, sed_file)).T # Angstrom, nJy
    if wavelength[0] == 0:
        wavelength, flux = wavelength[1:], flux[1:]
    frequencies, flux = spectral(wavelength*u.AA), flux*1e-9*u.Jy
    for j, z in enumerate(tqdm(z_range, desc=f"Redshifts for {galaxy_type} Type Galaxy")):
        for k, (transmission_profile, a, b) in enumerate(zip(transmission_profiles, a_values, b_values)):
            magnitudes[i][j][k] = magnitude(SourceSpectrum(Empirical1D, points=frequencies, lookup_table=flux, z=z, z_type="conserve_flux"), transmission_profile, a, b)

Redshifts for SB3 Type Galaxy:   0%|          | 0/350 [00:00<?, ?it/s]

/var/folders/8n/sm5x42hd1qlcn9q_5fqqkdfc0000gn/T/ipykernel_70788/1791165288.py:20: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  return 2.5*np.log10(quad(lambda f: transmission_profile(f).value * 3631 / (h.value  * f), a, b)[0]/quad(lambda f: transmission_profile(f).value * sed(f).value / (h.value * f), a, b)[0]) # m_AB
Redshifts for SB3 Type Galaxy:   1%|          | 3/350 [00:58<1:51:56, 19.36s/it]/var/folders/8n/sm5x42hd1qlcn9q_5fqqkdfc0000gn/T/ipykernel_70788/1791165288.py:20: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tole

KeyboardInterrupt: 